In [1]:
import pandas as pd
import json
import re
import requests
import numpy as np

In [2]:
# Get query results from uniprot (for UNIPROT ID —> sequence)
with open('20Nov2023uniprot_chkpoint_for_brenda.txt', 'r') as f:
    queried = f.readlines()

In [3]:
print (len(queried))
queried = [x for x in queried if 'rror' not in x]
print (len(queried))

142313
141515


In [4]:
def extract_header_information(header):
    regex_pattern = r'>?([^|]+)\|([^|]+)\|([^ ]+?) (.*?)(?: OS=(.+?))(?: OX=(.+))?(?: GN=(.+?))?(?: PE=(.+?))(?: SV=(.+?))'
    match = re.match(regex_pattern, header)
    if match:
        db_value = match.group(1)
        unique_identifier = match.group(2)
        entry_name = match.group(3)
        protein_name = match.group(4)
        organism_name = match.group(5)
        organism_identifier = match.group(6)
        gene_name = match.group(7)
        protein_existence = match.group(8)
        sequence_version = match.group(9)

        return {
            "db": db_value,
            "UniqueIdentifier": unique_identifier,
            "EntryName": entry_name,
            "ProteinName": protein_name,
            "OrganismName": organism_name,
            "OrganismIdentifier": organism_identifier,
            "GeneName": gene_name,
            "ProteinExistence": protein_existence,
            "SequenceVersion": sequence_version
        }
    else:
        print ("Failed parsing")
        return None

header_example = '>sp|Q01740|FMO1_HUMAN Flavin-containing monooxygenase 1 OS=Homo sapiens OX=9606 PE=1 SV=3'


result = extract_header_information(header_example)
if result:
    print("Extracted Information:")
    for key, value in result.items():
        print(f"{key}: {value}")
else:
    print("Header format doesn't match the pattern.")


def parse_uniprot_result (uniprot_result):
    header = uniprot_result.split('\n')[0]
    sequence = ''.join(uniprot_result.split('\n')[1:])
    info = extract_header_information(header)
    if info is not None:
        info['sequence'] = sequence
    else:
        print (header)
        print (sequence)
    return info

def parse_uniprot_batch (uniprot_batch):
    result_ls = []
    for uniprot_result in uniprot_batch.split('\n>'):
        if uniprot_result is not None:
            parsed = parse_uniprot_result(uniprot_result)
            if parsed is not None:
                result_ls.append(parsed)
    return result_ls

Extracted Information:
db: sp
UniqueIdentifier: Q01740
EntryName: FMO1_HUMAN
ProteinName: Flavin-containing monooxygenase 1
OrganismName: Homo sapiens
OrganismIdentifier: 9606
GeneName: None
ProteinExistence: 1
SequenceVersion: 3


In [5]:
parsed_queried = parse_uniprot_batch(''.join(queried))

In [6]:
# get previously parsed dictionary of UNIPROT ID —> sequence 
with open('protein_id_to_sequence.json', 'r') as f:
    pid2seq = json.load(f)

In [7]:
# Add BRENDA UNIPROT IDs to dictionary
for entry in parsed_queried:
    pid2seq[entry['UniqueIdentifier']] = entry['sequence']

In [8]:
# Save more complete dictionary
with open('20Nov2023_protein_id_to_sequence.json','w') as f:
    json.dump(pid2seq, f)

In [9]:
# Get info linking brenda reactions to UNIPROT IDs
with open('../parse_reaction_dbs/brenda/scraped_brenda_substrate_results.json', 'r') as f:
    #requires further conversion from uniprot to seq
    brenda2uniprot = json.load(f)

In [10]:
# clean brenda dict
# remove lines with no UNIPROT ID
cleaned_brenda2uniprot = {}
for ec in brenda2uniprot:
    new_entries = []
    for entry in brenda2uniprot[ec]:
        if entry['UNIPROT']!='-' and len(entry['UNIPROT']) and entry['SUBSTRATE']!='additional information'and entry['PRODUCT']!='?':
            new_entry = entry
            new_entry['molecules'] = set(entry['PRODUCT'].split(' + ')).union(set(entry['SUBSTRATE'].split(' + ')))
            new_entries.append(new_entry)
    cleaned_brenda2uniprot[ec] = new_entries

In [11]:
bkms = pd.read_csv('bkms/1Sep2023_bkms-mapped.txt', sep='\t').drop(columns=['Unnamed: 0.1','index','Unnamed: 0'])

In [12]:
brenda_subset = bkms.dropna(subset=['Reaction_ID_BRENDA'])
print (len (brenda_subset))

13731


In [13]:
def get_reactants (reaction):
    if '<=>' in reaction:
        return reaction.split(' <=> ')[0].split(' + ')
    elif ' = ' in reaction:
        return reaction.split(' = ')[0].split(' + ')
    else:
        raise 
def get_products (reaction):
    if '<=>' in reaction:
        return reaction.split(' <=> ')[1].split(' + ')
    elif ' = ' in reaction:
        return reaction.split(' = ')[1].split(' + ')
    else:
        raise 
        
brenda_subset['reactants'] = brenda_subset['Reaction'].map(lambda x : get_reactants(x))
brenda_subset['products'] = brenda_subset['Reaction'].map(lambda x : get_products(x))
brenda_subset['molecules'] = brenda_subset['Reaction'].map(lambda x : set(get_products(x)).union(set(get_reactants(x))) )

/var/folders/cb/fq3bwnj51bv2d7d694kx77pm0000gp/T/ipykernel_86597/1937314485.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  brenda_subset['reactants'] = brenda_subset['Reaction'].map(lambda x : get_reactants(x))
/var/folders/cb/fq3bwnj51bv2d7d694kx77pm0000gp/T/ipykernel_86597/1937314485.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  brenda_subset['products'] = brenda_subset['Reaction'].map(lambda x : get_products(x))
/var/folders/cb/fq3bwnj51bv2d7d694kx77pm0000gp/T/ipykernel_86597/1937314485.py:1

In [14]:
idx2uniprot = {}
no_ec = []
empty_ec = []
for idx in brenda_subset.index:
    ec =  brenda_subset.loc[idx,'EC_Number']
    if ec =='SPONTANEOUS':
        idx2uniprot[idx] = 'SPONTANEOUS'
    else:    
        if ec in cleaned_brenda2uniprot.keys():
            brenda_ec_page = cleaned_brenda2uniprot[ec]
            overlaps = []
            if len(brenda_ec_page):
                for entry in brenda_ec_page:
                    overlap = len(entry['molecules'].intersection(brenda_subset.loc[idx,'molecules']))
                    overlaps.append(overlap)
                
                idx2uniprot[idx] = brenda_ec_page[np.argmax(overlaps)]['UNIPROT']
            else:
                empty_ec.append(ec)

In [15]:
idx2seq = {}
for k,pid in idx2uniprot.items():
    
    if pid == 'SPONTANEOUS':
        idx2seq[k] = 'SPONTANEOUS'
        
    uniprot_options = [x.strip() for x in pid.replace(';',',').split(',')]
    
    for uid in uniprot_options:
        if uid in pid2seq.keys():
            idx2seq[k] = pid2seq[uid]

In [16]:
len(idx2seq)

10225

In [17]:
bkms['sequence'] = [idx2seq[idx] if idx in idx2seq.keys() else None for idx in bkms.index]

In [18]:
bkms['sequence'].dropna()

1        MEPEVIRDKDSMRKWSRAMRSQGKTIGLVPTMGYLHEGHLSLVRQS...
2        MHTSTTSTVPLEPWTAQQLQQATQGYWHKDQIPQTEIKRILTDSRH...
3        MRVALLMGGVSREREISLRSGERVKKALEKLGYEHTVFDVREDFLK...
4        MISVDRLSEEQALGMKEQEWAGPEALCPGWQEEEVSDGEGPEDSGH...
5        MSAQSAASRWTNLAGIGSCHPRHHHYPTSTKEVQDAFELVRSQNGK...
                               ...                        
25324    MREIIERVKEKTTIPVYERTIENVLSAIQASGDVWRIVDLSEEPLP...
25325    MREIIERVKEKTTIPVYERTIENVLSAIQASGDVWRIVDLSEEPLP...
25328    MREIIERVKEKTTIPVYERTIENVLSAIQASGDVWRIVDLSEEPLP...
25331    MGYNEQERAFIEWYPRGYGVGFKVKRRLFETQTEYQRLEIYETEGF...
25332    MDKLISNNKLKLSVVLLGGLCSLAYYHLKNKFHLSQFCFSKKWFSE...
Name: sequence, Length: 10225, dtype: object

## KEGG

In [19]:
with open('../parse_reaction_dbs/kegg/kegg_rids_to_sequences.json', 'r') as f:
    kegg2seq = json.load(f)

In [20]:
# Prefer KEGG sequence over BRENDA-derived sequence
for idx in bkms[~bkms['Reaction_ID_KEGG'].isna()].index:
    kegg_ids = bkms.loc[idx, 'Reaction_ID_KEGG'].replace(';',',').split(',')
    for kid in kegg_ids:
        if  kid in kegg2seq.keys():
            kegg_entry = kegg2seq[kid]
            if kegg_entry:
                bkms.loc[idx,'sequence'] = kegg_entry['sequence']

In [21]:
bkms['sequence'].dropna()

0        MKTAAQYEESLRRLNLKVYLQGERVENPVDHPIIRPSMNSVKTTYA...
1        MLIAHSIADLRSALASRGRPAFVPTMGNLHEGHLSLVRQARALGDV...
2        MSGTPMLTLQQAHALVQARIPQARLVGGPDALQLPLGRVHTDTRTL...
3        MSGTPMLTLQQAHALVQARIPQARLVGGPDALQLPLGRVHTDTRTL...
4        MLLCLSPAWLMKVAVPGQEGEASLLVSKAVSFYPGGLTFLDDFVPP...
                               ...                        
25328    MREIIERVKEKTTIPVYERTIENVLSAIQASGDVWRIVDLSEEPLP...
25329    MTTLPATSLSETARRSPERAVETYVEGRPYDLFYLSSAGRRLVGRK...
25330    MWVDCATGGGKARWPASRKIAGLTAAAPLPSPPQPAPRPLDIALLA...
25331    MWVDCATGGGKARWPASRKIAGLTAAAPLPSPPQPAPRPLDIALLA...
25332    MWVDCATGGGKARWPASRKIAGLTAAAPLPSPPQPAPRPLDIALLA...
Name: sequence, Length: 13708, dtype: object

In [22]:
bkms['Remark'].drop_duplicates()

0                                       natural substrates
12                 natural substrates, multi-step reaction
16                             natural substrates, generic
79                    natural substrates, generic, protein
170      natural substrates, multi-step reaction, gener...
232                            natural substrates, polymer
281                            natural substrates, protein
455                         natural substrates, nucleotide
493                   natural substrates, generic, polymer
1428      natural substrates, multi-step reaction, generic
1770         natural substrates, generic, polymer, protein
12267              natural substrates, generic, nucleotide
13851                 natural substrates, polymer, protein
Name: Remark, dtype: object

## MetaCyc

In [36]:
metacyc2seq = pd.read_csv('21Nov2023_metacyc_reactions_with_sequences.csv')

In [37]:
metacyc2seq_dict = pd.Series(metacyc2seq['sequence'].values, index = metacyc2seq['REACTION_ID'].values).to_dict()

In [39]:
for idx in bkms[~bkms['Reaction_ID_MetaCyc'].isna()].index:
    metacyc_ids = bkms.loc[idx, 'Reaction_ID_MetaCyc'].replace(';',',').split(',')
    for mid in metacyc_ids:
        if  bkms.loc[idx, 'Reaction_ID_MetaCyc'] in metacyc2seq_dict.keys():
            bkms.loc[idx,'sequence'] = metacyc2seq_dict[bkms.loc[idx, 'Reaction_ID_MetaCyc']]

In [40]:
bkms['sequence'].dropna()

0        MLMTAEQYIESLRKLNTRVYMFGEKIENWVDHPMIRPSINCVAMTY...
1        MKTETTIQGLAASLNPARAARKIIGFVPTMGNLHEGHLTLVREAKK...
2        MIPLGLGEIAEIVGGKATGESVTVTAPAVLDSRQAEPGGLFVAFAG...
3        MIPLGLGEIAEIVGGKATGESVTVTAPAVLDSRQAEPGGLFVAFAG...
4        MISVDRLSEEQALGMKEQEWAGPEALCPGWQEEEVSDGEGPEDSGH...
                               ...                        
25328    MREIIERVKEKTTIPVYERTIENVLSAIQASGDVWRIVDLSEEPLP...
25329    MTTLPATSLSETARRSPERAVETYVEGRPYDLFYLSSAGRRLVGRK...
25330    MAEKKQWHETLHDQFGQYFAVDNVLYHEKTDHQDLIIFENAAFGRV...
25331    MAEKKQWHETLHDQFGQYFAVDNVLYHEKTDHQDLIIFENAAFGRV...
25332    MAEKKQWHETLHDQFGQYFAVDNVLYHEKTDHQDLIIFENAAFGRV...
Name: sequence, Length: 18109, dtype: object

In [41]:
bkms[bkms['EC_Number'].map(lambda x: ',' in str(x))]

,ID,EC_Number,Recommended_Name,Reaction,Reaction_ID_BRENDA,Reaction_ID_KEGG,Reaction_ID_MetaCyc,Reaction_ID_SABIO_RK,BRENDA_Pathway_Name,KEGG_Pathway_ID,...,Missing_Substrate,Missing_Product,Commentary_KEGG,Commentary_MetaCyc,Remark,smiles,num products,mapped_smiles,num_products,sequence


In [42]:
np.random.seed(4)
ridxs = np.random.choice(bkms[bkms['sequence'].isna()].index, 5)

bkms.loc[ridxs,:]

,ID,EC_Number,Recommended_Name,Reaction,Reaction_ID_BRENDA,Reaction_ID_KEGG,Reaction_ID_MetaCyc,Reaction_ID_SABIO_RK,BRENDA_Pathway_Name,KEGG_Pathway_ID,...,Missing_Substrate,Missing_Product,Commentary_KEGG,Commentary_MetaCyc,Remark,smiles,num products,mapped_smiles,num_products,sequence
5288,23003,NaN,NaN,"Bromobenzene-2,3-oxide <=> 2-Bromophenol",NaN,R07074,NaN,NaN,NaN,rn00980,...,NaN,NaN,NaN,NaN,natural substrates,BrC1=CC=CC2OC12>>Oc1ccccc1Br,1,[Br:1][C:2]1=[CH:3][CH:4]=[CH:5][CH:6]2[O:7][...,1,None
1871,12710,1.7.1.9,nitroquinoline-N-oxide reductase,4-(hydroxyamino)quinoline N-oxide + 2 NAD+ + H...,BR36929_WOP,R04228,1.7.1.9-RXN_WOP,NaN,NaN,NaN,...,NaN,NaN,NADPH (see R04229),NaN,natural substrates,O/N=c1/ccn(O)c2ccccc12>>O=[N+]([O-])c1cc[n+]([...,1,[OH:1]/[N:2]=[c:3]\1/[cH:4][cH:5][n:6]([OH:7]...,1,None
7774,28067,2.7.1.202,protein-Npi-phosphohistidine-D-fructose phosph...,[HPr protein]-Npi-phospho-L-histidine + keto-D...,NaN,NaN,RXN-15150,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"natural substrates, protein",O=C(CO)[C@@H](O)[C@H](O)[C@H](O)CO>>O=C(COP(=O...,1,[O:1]=[C:2]([CH2:3][OH:4])[C@@H:5]([OH:6])[C@...,1,None
4520,21683,2.7.7.28,nucleoside-triphosphate-aldose-1-phosphate nuc...,GTP + alpha-D-Hexose 1-phosphate <=> Diphospha...,NaN,R03318,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"natural substrates, generic",N=c1[nH]c(=O)c2ncn([C@@H]3O[C@H](CO[PH](=O)(=O...,1,[O:1]=[c:2]1[nH:3][c:4](=[NH:5])[nH:6][c:7]2[...,1,None
21194,23976,4.1.1,NaN,"Phenanthrene-4,5-dicarboxylate <=> Phenanthren...",NaN,R09164,NaN,NaN,NaN,rn00624; rn01120,...,NaN,NaN,"phenanthrene-4,5-dicarboxylate decarboxylase",NaN,natural substrates,O=C(O)c1cccc2ccc3ccccc3c12>>O=C(O)c1cccc2ccc3c...,1,[O:1]=[C:2]([OH:3])[c:4]1[cH:5][cH:6][cH:7][c:...,1,None


In [35]:
bkms.to_csv('bkms/21Nov2023_bkms-mapped_w_seqs.tsv', sep='\t', index=False)